# Gemini Text Summarization Walkthrough

This notebook shows how to use Gemini for summarization and how the FastAPI app works.


## 1. Setup

Make sure your virtual environment is active and dependencies are installed:

```
pip install -r requirements.txt
```

Set your API key in the environment:

```
$env:GEMINI_API_KEY="YOUR_API_KEY"
```


In [ ]:
import os
import google.generativeai as genai

api_key = os.getenv("GEMINI_API_KEY", "").strip()
if not api_key:
    raise RuntimeError("GEMINI_API_KEY is not set")

genai.configure(api_key=api_key)
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config={
        "temperature": 0.2,
        "top_p": 0.8,
        "top_k": 64,
        "max_output_tokens": 1024,
    },
)


## 2. Simple Summarization
We pass the text to Gemini with a concise summary instruction.


In [ ]:
def summarize(text, max_words=120, style="concise"):
    if style == "bullet points":
        style_text = "Return bullet points, each line starting with '- '."
    elif style == "numbered list":
        style_text = "Return a numbered list, one item per line like '1. ...'."
    else:
        style_text = f"Style: {style}."

    prompt = (
        "You are a helpful assistant that summarizes text.\n"
        f"Max {max_words} words. {style_text}\n"
        "Return only the summary.\n\n"
        f"TEXT:\n{text}"
    )
    response = model.generate_content(prompt)
    return (response.text or "").strip()

sample_text = (
    "FastAPI is a modern, fast web framework for building APIs with Python. "
    "It is based on standard Python type hints and provides automatic "
    "interactive documentation using OpenAPI. Developers use FastAPI "
    "to build reliable, performant services."
)

summarize(sample_text, max_words=50, style="executive")


## Styles (Same as API Dropdown)
Choose a style to control the summary format and tone.

Available styles:
- `concise`
- `executive`
- `bullet points` (forces '- ' bullets)
- `numbered list` (forces '1.', '2.' format)
- `simple`
- `technical`
- `highlights`
- `action items`
- `key takeaways`
- `tl;dr`


## 3. Bullet / Numbered Summary
Use `bullet points` or `numbered list` for structured output.


In [ ]:
summarize(sample_text, max_words=60, style="bullet points")
# summarize(sample_text, max_words=60, style="numbered list")


## 4. How This Maps to the FastAPI App
The FastAPI endpoint in `app.py` does the same thing:
- Validates input with Pydantic
- Builds the prompt
- Calls Gemini
- Returns a JSON response with summary + metadata


In [ ]:
# Example request body used by the API
example_request = {
    "text": sample_text,
    "max_words": 80,
    "style": "concise"
}
example_request
